In [1]:
from alquimodelia import alquimodelia as alqm
import importlib 
importlib.reload(alqm)

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


<module 'alquimodelia.alquimodelia' from '/home/joao/Documents/gits/alquimodelia/alquimodelia/alquimodelia.py'>

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2022-08-08 14:05:43.932097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-08 14:05:44.001948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 14:05:44.019892: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 14:05:44.020140: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2022-08-08 14:05:45.835942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 14:05:45.836464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 14:05:45.836809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
pixel_model = alqm.ModelMagia('pixel',
    timesteps=12,
    width=None,
    height=None,
    padding=None,
    num_bands=10,
    num_classes=3,
    activation_final=None)

2022-08-08 15:15:47.566399: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 15:15:47.578987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 15:15:47.579534: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 15:15:47.581263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
pixel_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12, 10)]     0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 12, 10)      40          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv1d (Conv1D)                (None, 10, 64)       1984        ['batch_normalization[0][0]']    
                                                                                                  
 conv1d_2 (Conv1D)              (None, 8, 64)        3264        ['batch_normalization[0][0]']

In [4]:
# From https://github.com/karolzak/keras-unet
VUNET_model = alqm.ModelMagia('keras_unet.custom_unet',
    timesteps=12,
    width=512,
    height=512,
    padding=None,
    num_bands=10,
    num_classes=3,
    activation_final=None)

In [5]:
VUNET_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                0)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  5760        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 rmalization)                   )                                                           

 conv2d_9 (Conv2D)              (None, 32, 32, 1024  9437184     ['spatial_dropout2d_4[0][0]']    
                                )                                                                 
                                                                                                  
 batch_normalization_15 (BatchN  (None, 32, 32, 1024  4096       ['conv2d_9[0][0]']               
 ormalization)                  )                                                                 
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 64, 64, 512)  2097664    ['batch_normalization_15[0][0]'] 
 ose)                                                                                             
                                                                                                  
 concatenate_1 (Concatenate)    (None, 64, 64, 1024  0           ['conv2d_transpose[0][0]',       
          